<a href="https://colab.research.google.com/github/MatiasJJ/ATML/blob/edits/Ex2/Ex2_Matias_Jakub_Madeleine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ex2

*by: Madeleine Ekblom, Matias Jääskeläinen, Jakub Kubečka*

Time used: 3h

In [0]:
import numpy as np
#import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#import statsmodels.api as sm

JK: I have to say that the performance of this Google Jupyter is not superfast but at least we can work together.

In [0]:
np.random.seed(42)
np.random.normal(0,1,10)

array([ 0.49671415, -0.1382643 ,  0.64768854,  1.52302986, -0.23415337,
       -0.23413696,  1.57921282,  0.76743473, -0.46947439,  0.54256004])